In [ ]:
from ocpmodels.datasets import SinglePointLmdbDataset
import pandas as pd
import numpy as np

In [ ]:
def splmdbdataset_to_dicts(dataset):
    array = []
    for obj in dataset:
        d = {}
        d['id'] = obj['sid']
        d['y_relaxed'] = obj['y_relaxed']
        array.append(d)
    return array

In [ ]:
dataset_val_ood_both = SinglePointLmdbDataset({"src": "../../ocp_datasets/data/is2re/all/val_ood_both/data.lmdb"})

In [ ]:
list_of_dicts = splmdbdataset_to_dicts(dataset_val_ood_both)

In [ ]:
dataframe_val_ood_both = pd.DataFrame(list_of_dicts)

In [ ]:
predicted_val_ood_both = np.load('../results/2021-08-25-11-23-28/is2re_predictions.npz')

In [ ]:
predicted_val_ood_both = pd.DataFrame({'id' : predicted_val_ood_both['ids'], 'y_pred' : predicted_val_ood_both['energy']})

In [ ]:
predicted_val_ood_both['id'] = predicted_val_ood_both['id'].apply(int)

In [ ]:
pivot = dataframe_val_ood_both.merge(predicted_val_ood_both, left_on='id', right_on='id')

In [ ]:
pivot['abs_error'] = abs(pivot['y_relaxed'] - pivot['y_pred'])

In [ ]:
mae = pivot['abs_error'].mean()
mae

In [ ]:
pivot[pivot['abs_error'] < 0.02]

In [ ]:
pivot['treshold'] = pivot['abs_error'] > 8
pivot['treshold'].mean()

In [ ]:
pivot['abs_error'].hist(bins=1000, figsize=(15, 10))

In [ ]:
structures_pivot = pivot[pivot['treshold'] == 1]

In [ ]:
oc20_data_mapping_df = pd.DataFrame(data = pd.read_pickle('../../ocp_datasets/oc20_data_mapping.pkl'))

In [ ]:
oc20_data_mapping_df.T

In [ ]:
structures = {}
for structure in structures_pivot['id']:
    structure_id = 'random' + str(structure)
    structures[structure_id] = oc20_data_mapping[structure_id]

oc20_data_mapping_fields = {
            "bulk_mpid" : "Materials Project ID of the bulk system used corresponding the the catalyst surface",
            "bulk_symbols":  "Chemical composition of the bulk counterpart",
            "ads_symbols":  "Chemical composition of the adsorbate counterpart",
            "ads_id": "internal unique identifier, one for each of the 82 adsorbates used in the dataset",
            "bulk_id": "internal unique identifier one for each of the 11500 bulks used in the dataset",
            "miller_index": "3-tuple of integers indicating the Miller indices of the surface",
            "shift": "c-direction shift used to determine cutoff for the surface (c-direction is following the nomenclature from Pymatgen)",
            "top": "boolean indicating whether the chosen surface was at the top or bottom of the originally enumerated surface",
            "adsorption_site": "A tuple of 3-tuples containing the Cartesian coordinates of each binding adsorbate atom"
}

oc20_data_mapping_columns = [i for i in oc20_data_mapping_fields.keys()]

In [ ]:
structures_df = pd.DataFrame(data = structures)
structures_df.T

In [ ]:
structures_df.T.to_csv("structures_df.csv")